[[651170.3]]


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import json
import ast


In [2]:
#loading additional data stored in json column into dataframe

# file_name = "all_housing_data.csv"



# df = pd.read_csv(file_name)

# ad = pd.DataFrame()


# for i, j in df.iterrows():
    
    
#     if i == 0:
#         a_json = json.loads(j['Additional_fields'])
        
#         data = ast.literal_eval(a_json)
#         a = list(data.items())
        
#         additional_df = pd.DataFrame(a)
#         additional_df = additional_df.T
        
#     else:

#         a_json = json.loads(j['Additional_fields'])
#         data = ast.literal_eval(a_json)
#         a = list(data.values())
#         new_row = pd.DataFrame(a).T
#         additional_df = additional_df.append(new_row)
        
        
    


# new_header = additional_df.iloc[0] 
# additional_df = additional_df[1:] 
# additional_df.columns = new_header 


# additional_df


In [3]:
# list(additional_df)


In [4]:
data = pd.read_csv("dataset_final.csv")
data

,Sold_price,Squarefootage,Type,Style,Community,Municipality District,longitude,latitude,Bedrooms,Dens,Bathrooms,Kitchens,Rooms,Parking Total
0,205000,950.0,Condo Apartment,Apartment,Black Creek,W05,-79.519841,43.769643,2,0,1,1,5,1.0
1,230000,550.0,Co-op / Co-Ownership Apartment,Apartment,West Hill,E10,-79.182497,43.775476,1,0,1,1,4,0.0
2,299000,1100.0,Condo Townhouse,Stacked Townhouse,Glenfield-Jane Heights,W05,-79.515943,43.749223,3,0,2,1,6,1.0
3,275000,250.0,Condo Apartment,Apartment,Downsview-Roding-CFB,W05,-79.481416,43.725043,0,0,1,1,3,1.0
4,305888,1100.0,Condo Apartment,Apartment,Glenfield-Jane Heights,W05,-79.515943,43.749223,3,0,2,1,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6212,6198000,4250.5,Detached,2-Storey,Forest Hill South,C03,-79.418061,43.696341,3,2,5,1,11,3.0
6213,7000000,5000.0,Detached,2-Storey,Forest Hill South,C03,-79.417288,43.691219,4,2,8,1,9,6.0
6214,7150000,5000.0,Detached,2-Storey,Bridle Path-Sunnybrook-York Mills,C12,-79.390151,43.743789,4,1,8,1,10,7.0
6215,6800000,5000.0,Detached,2-Storey,Forest Hill South,C03,-79.415512,43.700019,4,1,7,1,9,6.0


In [5]:
# defining a callback function for early stopping of model
def get_callbacks():
  return [
    tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', patience=50, verbose=1),
  ]

In [6]:
# Creating custom R^2 performance metric for keras models
def r2(y_true,y_pred):
    from keras import backend as K
    SS_res = K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

In [7]:



X = data.drop(['Sold_price','longitude','latitude','Community','Municipality District'],axis=1)
Y = data.drop(['Squarefootage','Type','Style','Community','Municipality District','Bedrooms','Dens','Bathrooms','Kitchens','Rooms','Parking Total','longitude','latitude'],axis=1)




In [8]:
Y

,Sold_price
0,205000
1,230000
2,299000
3,275000
4,305888
...,...
6212,6198000
6213,7000000
6214,7150000
6215,6800000


In [9]:
X

,Squarefootage,Type,Style,Bedrooms,Dens,Bathrooms,Kitchens,Rooms,Parking Total
0,950.0,Condo Apartment,Apartment,2,0,1,1,5,1.0
1,550.0,Co-op / Co-Ownership Apartment,Apartment,1,0,1,1,4,0.0
2,1100.0,Condo Townhouse,Stacked Townhouse,3,0,2,1,6,1.0
3,250.0,Condo Apartment,Apartment,0,0,1,1,3,1.0
4,1100.0,Condo Apartment,Apartment,3,0,2,1,6,1.0
...,...,...,...,...,...,...,...,...,...
6212,4250.5,Detached,2-Storey,3,2,5,1,11,3.0
6213,5000.0,Detached,2-Storey,4,2,8,1,9,6.0
6214,5000.0,Detached,2-Storey,4,1,8,1,10,7.0
6215,5000.0,Detached,2-Storey,4,1,7,1,9,6.0


In [10]:


d = {}
print(len(X.columns))

for i in range(len(X.columns)):
    #if column is categorical
    if type(X.iloc[0,i])==str:
#         print("categorical")
#         print(X.iloc[0,i])

        le = LabelEncoder()
        X[X.columns[i]] = le.fit_transform(X[X.columns[i]])
        
        print(le.classes_)
        
        print(X[X.columns[i]])
        series = X[X.columns[i]]
        
        d[series.name] = {}
        
        le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        
        d[series.name] = le_name_mapping
        
        
    
    else:
        print()
#         p = X[X.columns[i]]
        
#         print(p)
#         if p.min()==p.max():
#             continue
#         p -= p.min()
#         p /= p.max()
#         X[X.columns[i]] = p


print(d)  

9

['Co-op / Co-Ownership Apartment' 'Condo Apartment' 'Condo Townhouse'
 'Det Condo' 'Detached' 'Multiplex' 'Other' 'Semi-Detached'
 'Store W/Apt/Offc' 'Townhouse']
0       1
1       0
2       2
3       1
4       1
       ..
6212    4
6213    4
6214    4
6215    4
6216    4
Name: Type, Length: 6217, dtype: int64
['1 1/2 Storey' '2 1/2 Storey' '2-Storey' '3-Storey' 'Apartment'
 'Bachelor/Studio' 'Backsplit' 'Bungalow' 'Loft' 'Sidesplit'
 'Stacked Townhouse']
0        4
1        4
2       10
3        4
4        4
        ..
6212     2
6213     2
6214     2
6215     2
6216     2
Name: Style, Length: 6217, dtype: int64






{'Type': {'Co-op / Co-Ownership Apartment': 0, 'Condo Apartment': 1, 'Condo Townhouse': 2, 'Det Condo': 3, 'Detached': 4, 'Multiplex': 5, 'Other': 6, 'Semi-Detached': 7, 'Store W/Apt/Offc': 8, 'Townhouse': 9}, 'Style': {'1 1/2 Storey': 0, '2 1/2 Storey': 1, '2-Storey': 2, '3-Storey': 3, 'Apartment': 4, 'Bachelor/Studio': 5, 'Backsplit': 6, 'Bungalow': 7, 'Loft': 8, 'S

In [11]:
yScaler = StandardScaler()
yScaler.fit(Y)
Y = yScaler.transform(Y)





In [12]:
Xtr,Xts,Ytr,Yts = train_test_split(X,Y,test_size=0.1, random_state=42)


In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=[X.shape[1]]),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(1)
])

2021-09-06 18:41:18.877065: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-06 18:41:18.877377: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_squared_error'])

In [15]:
model.fit(X, Y, epochs=500)

2021-09-06 18:41:22.594284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/500
195/195 [==============================] - 0s 529us/step - loss: 10422.0059 - mean_squared_error: 10422.0059
Epoch 2/500
195/195 [==============================] - 0s 482us/step - loss: 1263.7076 - mean_squared_error: 1263.7076
Epoch 3/500
195/195 [==============================] - 0s 487us/step - loss: 265.2017 - mean_squared_error: 265.2017
Epoch 4/500
195/195 [==============================] - 0s 486us/step - loss: 45.6552 - mean_squared_error: 45.6552
Epoch 5/500
195/195 [==============================] - 0s 484us/step - loss: 13.9505 - mean_squared_error: 13.9505
Epoch 6/500
195/195 [==============================] - 0s 485us/step - loss: 5.0732 - mean_squared_error: 5.0732
Epoch 7/500
195/195 [==============================] - 0s 465us/step - loss: 2.0148 - mean_squared_error: 2.0148
Epoch 8/500
195/195 [==============================] - 0s 498us/step - loss: 1.6451 - mean_squared_error: 1.6451
Epoch 9/500
195/195 [==============================] - 0s 480us/step - los

195/195 [==============================] - 0s 462us/step - loss: 0.3752 - mean_squared_error: 0.3752
Epoch 144/500
195/195 [==============================] - 0s 446us/step - loss: 0.3959 - mean_squared_error: 0.3959
Epoch 145/500
195/195 [==============================] - 0s 445us/step - loss: 0.3253 - mean_squared_error: 0.3253
Epoch 146/500
195/195 [==============================] - 0s 446us/step - loss: 0.3575 - mean_squared_error: 0.3575
Epoch 147/500
195/195 [==============================] - 0s 440us/step - loss: 0.4661 - mean_squared_error: 0.4661
Epoch 148/500
195/195 [==============================] - 0s 442us/step - loss: 0.3956 - mean_squared_error: 0.3956
Epoch 149/500
195/195 [==============================] - 0s 444us/step - loss: 0.3272 - mean_squared_error: 0.3272
Epoch 150/500
195/195 [==============================] - 0s 451us/step - loss: 0.4162 - mean_squared_error: 0.4162
Epoch 151/500
195/195 [==============================] - 0s 456us/step - loss: 0.3937 - mean_s

195/195 [==============================] - 0s 461us/step - loss: 0.3371 - mean_squared_error: 0.3371
Epoch 286/500
195/195 [==============================] - 0s 458us/step - loss: 0.3854 - mean_squared_error: 0.3854
Epoch 287/500
195/195 [==============================] - 0s 453us/step - loss: 0.3386 - mean_squared_error: 0.3386
Epoch 288/500
195/195 [==============================] - 0s 465us/step - loss: 0.3060 - mean_squared_error: 0.3060
Epoch 289/500
195/195 [==============================] - 0s 454us/step - loss: 0.3740 - mean_squared_error: 0.3740
Epoch 290/500
195/195 [==============================] - 0s 455us/step - loss: 0.3454 - mean_squared_error: 0.3454
Epoch 291/500
195/195 [==============================] - 0s 459us/step - loss: 0.3144 - mean_squared_error: 0.3144
Epoch 292/500
195/195 [==============================] - 0s 453us/step - loss: 0.3081 - mean_squared_error: 0.3081
Epoch 293/500
195/195 [==============================] - 0s 463us/step - loss: 0.3559 - mean_s

195/195 [==============================] - 0s 461us/step - loss: 0.3261 - mean_squared_error: 0.3261
Epoch 428/500
195/195 [==============================] - 0s 455us/step - loss: 0.3608 - mean_squared_error: 0.3608
Epoch 429/500
195/195 [==============================] - 0s 457us/step - loss: 0.3415 - mean_squared_error: 0.3415
Epoch 430/500
195/195 [==============================] - 0s 457us/step - loss: 0.3449 - mean_squared_error: 0.3449
Epoch 431/500
195/195 [==============================] - 0s 456us/step - loss: 0.3366 - mean_squared_error: 0.3366
Epoch 432/500
195/195 [==============================] - 0s 459us/step - loss: 0.3858 - mean_squared_error: 0.3858
Epoch 433/500
195/195 [==============================] - 0s 451us/step - loss: 0.3822 - mean_squared_error: 0.3822
Epoch 434/500
195/195 [==============================] - 0s 461us/step - loss: 0.3119 - mean_squared_error: 0.3119
Epoch 435/500
195/195 [==============================] - 0s 476us/step - loss: 0.3436 - mean_s

In [16]:
def prediction(rows):
    result = model.predict(rows)
    scaledRes = []
    for i in result:
        scaledRes += [yScaler.inverse_transform(i)]
    return scaledRes

In [17]:
# model.evaluate(Xts, Yts, batch_size=128, verbose =1)

In [18]:
x = np.array(X.iloc[0])

In [19]:
x

array([950.,   1.,   4.,   2.,   0.,   1.,   1.,   5.,   1.])

In [20]:
X.iloc[0]

Squarefootage    950.0
Type               1.0
Style              4.0
Bedrooms           2.0
Dens               0.0
Bathrooms          1.0
Kitchens           1.0
Rooms              5.0
Parking Total      1.0
Name: 0, dtype: float64

In [23]:
# print(d)  
# d = {1: {'name': 'John', 'age': '27', 'sex': 'Male'},
#           2: {'name': 'Marie', 'age': '22', 'sex': 'Female'}}


d = {'Type': {'Co-op / Co-Ownership Apartment': 0, 'Condo Apartment': 1, 'Condo Townhouse': 2, 'Det Condo': 3, 'Detached': 4, 'Multiplex': 5, 'Other': 6, 'Semi-Detached': 7, 'Store W/Apt/Offc': 8, 'Townhouse': 9}, 'Style': {'1 1/2 Storey': 0, '2 1/2 Storey': 1, '2-Storey': 2, '3-Storey': 3, 'Apartment': 4, 'Bachelor/Studio': 5, 'Backsplit': 6, 'Bungalow': 7, 'Loft': 8, 'Sidesplit': 9, 'Stacked Townhouse': 10}}

print(d)






def predict(sqft, property_type, style, bedrooms, dens, bathrooms, kitchens, rooms, parking):
    
    
    
    
    
    
    test_input = np.array([sqft, property_type, style, bedrooms, dens, bathrooms, kitchens, rooms, parking], dtype = 'object' )

    test_input = np.array([950, 'Condo Apartment', 'Apartment', 2, 0, 1,1, 5, 1.0 ], dtype = 'object' )


    test_input[1] = d['Type'][test_input[1]]
    test_input[2] = d['Style'][test_input[2]]



#     print(type(test_input))
    test_input = test_input.reshape(1,9)
#     print(test_input)

    test_input = np.asarray(test_input).astype(np.float32)


    load_model_from_file = tf.keras.models.load_model("./models/mvp_model")

    result = load_model_from_file.predict(test_input)
#     print(result)
    
    
    return yScaler.inverse_transform(result)

# print(yScaler.inverse_transform(result))


print(predict(950, 'Condo Apartment', 'Apartment', 2, 0, 1,1, 5, 1.0 ))


# print(test_input)
# print(test_input[1])
# print(type(test_input))


{'Type': {'Co-op / Co-Ownership Apartment': 0, 'Condo Apartment': 1, 'Condo Townhouse': 2, 'Det Condo': 3, 'Detached': 4, 'Multiplex': 5, 'Other': 6, 'Semi-Detached': 7, 'Store W/Apt/Offc': 8, 'Townhouse': 9}, 'Style': {'1 1/2 Storey': 0, '2 1/2 Storey': 1, '2-Storey': 2, '3-Storey': 3, 'Apartment': 4, 'Bachelor/Studio': 5, 'Backsplit': 6, 'Bungalow': 7, 'Loft': 8, 'Sidesplit': 9, 'Stacked Townhouse': 10}}
[[651170.3]]


In [188]:


x = np.array(X.iloc[0])

x = x.reshape(1,9)

print (x)
print(type(x))

[[950.   1.   4.   2.   0.   1.   1.   5.   1.]]
<class 'numpy.ndarray'>


In [185]:

model.save("./models/mvp_model")

load_model_from_file = tf.keras.models.load_model("./models/mvp_model")

result = load_model_from_file.predict(x)
print(result)
print(yScaler.inverse_transform(result))

INFO:tensorflow:Assets written to: ./models/mvp_model/assets
[[-0.401256]]
[[651170.3]]


In [23]:
#Setting and Training using a Hyperparameter Tuner

In [24]:
import keras_tuner as kt
from tensorflow import keras
from keras_tuner import Hyperband,RandomSearch
import keras_tuner
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_mean_absolute_error', patience=10, verbose=1)


def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=[X.shape[1]]))
    for i in range(hp.Int("num_layers", 2, 10)): #number of layers
        model.add(
            layers.Dense(
                units=hp.Int("units_" + str(i), min_value=11, max_value=512, step=32), #number of units
                activation="relu",
            )
        )
    model.add(keras.layers.Dense(1))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])),
                loss=keras.losses.mean_absolute_error,
              metrics=['mean_squared_error',r2,keras.losses.mean_absolute_error])
    return model

In [25]:
tuner = kt.Hyperband(model_builder,
                     objective=keras_tuner.Objective("val_r2", direction="max"),
                     max_epochs=1000,
                     executions_per_trial=3,
                     overwrite=True,
                     directory="my_dir",
                     project_name="KerasRegressor",
                    
)


    


In [26]:
tuner.search(Xtr, Ytr, epochs=1000, validation_split=0.2,callbacks=[callback])

Trial 61 Complete [00h 00m 04s]
val_r2: 0.22877928614616394

Best val_r2 So Far: 0.4829423725605011
Total elapsed time: 00h 04m 20s

Search: Running Trial #62

Hyperparameter    |Value             |Best Value So Far 
num_layers        |8                 |2                 
units_0           |331               |427               
units_1           |43                |75                
learning_rate     |0.0001            |0.01              
units_2           |107               |491               
units_3           |43                |203               
units_4           |235               |171               
units_5           |107               |235               
units_6           |363               |299               
units_7           |203               |299               
units_8           |107               |459               
units_9           |139               |11                
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0   

KeyboardInterrupt: 

In [36]:
# bestHparams=tuner.get_best_hyperparameters(num_trials=1)[0]

models_hyper = tuner.get_best_models(num_models=2)
best_model = models_hyper[0]
score = best_model.evaluate(Xts, Yts, verbose=0)

In [30]:
model = tuner.hypermodel.build(bestHparams)
model.fit(X, Y, epochs=50, validation_split=0.2)

Epoch 1/50
156/156 [==============================] - 1s 5ms/step - loss: 0.3045 - mean_squared_error: 0.1758 - r2: -1.9423 - mean_absolute_error: 0.3045 - val_loss: 1.7181 - val_mean_squared_error: 5.0592 - val_r2: -38.6979 - val_mean_absolute_error: 1.7181
Epoch 2/50
156/156 [==============================] - 1s 4ms/step - loss: 0.2012 - mean_squared_error: 0.0677 - r2: -0.0619 - mean_absolute_error: 0.2012 - val_loss: 1.6660 - val_mean_squared_error: 4.8625 - val_r2: -36.1137 - val_mean_absolute_error: 1.6660
Epoch 3/50
156/156 [==============================] - 1s 4ms/step - loss: 0.1746 - mean_squared_error: 0.0513 - r2: 0.1659 - mean_absolute_error: 0.1746 - val_loss: 1.5677 - val_mean_squared_error: 4.4545 - val_r2: -32.0261 - val_mean_absolute_error: 1.5677
Epoch 4/50
156/156 [==============================] - 1s 4ms/step - loss: 0.1678 - mean_squared_error: 0.0485 - r2: 0.2042 - mean_absolute_error: 0.1678 - val_loss: 1.6145 - val_mean_squared_error: 4.6135 - val_r2: -34.2416 

Epoch 33/50
156/156 [==============================] - 1s 4ms/step - loss: 0.1236 - mean_squared_error: 0.0278 - r2: 0.5546 - mean_absolute_error: 0.1236 - val_loss: 1.4232 - val_mean_squared_error: 4.0345 - val_r2: -26.2759 - val_mean_absolute_error: 1.4232
Epoch 34/50
156/156 [==============================] - 1s 4ms/step - loss: 0.1193 - mean_squared_error: 0.0264 - r2: 0.5765 - mean_absolute_error: 0.1193 - val_loss: 1.3972 - val_mean_squared_error: 3.9810 - val_r2: -25.3681 - val_mean_absolute_error: 1.3972
Epoch 35/50
156/156 [==============================] - 1s 4ms/step - loss: 0.1232 - mean_squared_error: 0.0280 - r2: 0.5391 - mean_absolute_error: 0.1232 - val_loss: 1.3369 - val_mean_squared_error: 3.6932 - val_r2: -23.2159 - val_mean_absolute_error: 1.3369
Epoch 36/50
156/156 [==============================] - 1s 4ms/step - loss: 0.1221 - mean_squared_error: 0.0278 - r2: 0.5382 - mean_absolute_error: 0.1221 - val_loss: 1.3691 - val_mean_squared_error: 3.8827 - val_r2: -24.456

In [31]:
model.evaluate(X, Y)

195/195 [==============================] - 0s 1ms/step - loss: 0.3633 - mean_squared_error: 0.8014 - r2: -13.1551 - mean_absolute_error: 0.3633


[0.36327672004699707,
 0.8013868927955627,
 -13.155065536499023,
 0.36327672004699707]

In [ ]:
def predict(sqft, property_type, style, bedrooms, dens, bathrooms, kitchens, rooms, parking):
    
    
    
    test_input = np.array([sqft, property_type, style, bedrooms, dens, bathrooms, kitchens, rooms, parking], dtype = 'object' )

    test_input = np.array([950, 'Condo Apartment', 'Apartment', 2, 0, 1,1, 5, 1.0 ], dtype = 'object' )


    test_input[1] = d['Type'][test_input[1]]
    test_input[2] = d['Style'][test_input[2]]



    test_input = test_input.reshape(1,9)


    test_input = np.asarray(test_input).astype(np.float32)


    load_model_from_file = tf.keras.models.load_model("./models/mvp_model")

    result = load_model_from_file.predict(test_input)

    
    
    return yScaler.inverse_transform(result)




print(predict(950, 'Condo Apartment', 'Apartment', 2, 0, 1,1, 5, 1.0 ))